# Para Google Colaboratory (Drive)

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

In [ ]:
#Deiv ruta
!ls "drive/My Drive/ML-20182/Notebooks/Training"

train_path = '/content/drive/My Drive/2018-2/Aprendiza de máquina/ML-20182/Notebooks/Training'
#!unzip -uq "drive/My Drive/ML-20182/Dataset&Papers/Dataset/fruits.zip" -d "drive/My Drive/ML-20182/Notebooks/Unzipeado/"

In [ ]:
#Jorge ruta
ls -1 | wc -l '/content/drive/My Drive/2018-2/Aprendiza de máquina/ML-20182/Notebooks/Training/'

train_path = '/content/drive/My Drive/2018-2/Aprendiza de máquina/ML-20182/Notebooks/Training'

# Para Github (Local)

In [1]:
#path
!ls ../data/

train_path_str = '../data/Training'
test_path_str = '../data/Test'
img_size = 200

LICENSE  readme.md  Test  test-multiple_fruits	Training


# Imágenes - Preprocesamiento

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm import tqdm_notebook
from pathlib import Path
%matplotlib inline

In [3]:
##Extrayendo las categorias

def extract_categories(path_str):
    path = Path(path_str)
    categories = []
    for index, e in enumerate(path.iterdir()):
        categories.append((index, e))
    return categories


In [ ]:
##Extrayendo las categorias (test)

categories_train = extract_categories(train_path_str)
categories_test = extract_categories(test_path_str)

print('Clases en Train: ', len(categories_train))
print('Clases en Test: ', len(categories_test))

In [ ]:
#Revisamos que está en orden

#train

categories_train

In [ ]:
#test
categories_test

In [ ]:
#Conteo de imágenes por Training y Testing

def count_images(categories):
    count = 0
    for i,path in categories:
        fruit_path = Path(path)
        for img_fruit in fruit_path.iterdir():
            count += 1
    return count


In [ ]:
# Revisamos cantidad de imagenes para cada set  (deberían ser 41322 y 13877, por info del repo de GitHub)
print('La cantidad de imágenes para  el train es ', count_images(categories_train))
print('La cantidad de imágenes para  el test es ', count_images(categories_test))


In [ ]:
# Tamaño del training set: 41322 imgs
train_size = count_images(categories_train)

# Tamaño del test set: 13877 imgs
test_size = count_images(categories_test)

# Primero inicializamos los arrays que vamos a usar
x_train = np.ndarray(shape=(train_size, img_size, img_size,3), dtype=np.float32)
y_train = np.zeros(shape=(train_size), dtype=np.int8)
x_val = np.ndarray(shape=(test_size, img_size, img_size,3), dtype=np.float32)
y_val = np.zeros(shape=(test_size), dtype=np.int8)

In [ ]:
# Definimos una funcion para leer una imagen y hacer el preprocesamiento
from keras.applications.resnet50 import preprocess_input


def read_img(path):
    x = Image.open(path)
    x = x.resize((img_size, img_size))
    x = np.asarray(x, np.float32)
    return preprocess_input(x)

In [ ]:
# Cargamos el train set
global_index = 0
for i,path in categories_train:
    print(i,path)
    
    train_path = Path(path)
    for index, e in enumerate(train_path.iterdir()):
      #print(global_index)
      x_train[global_index] = read_img(e)
      y_train[global_index] = i
      global_index += 1
print(len(x_train), len(y_train))

In [ ]:
# Cargamos el test set
global_index = 0
for i,path in categories_test:
    print(i,path)
    
    test_path = Path(path)
    imagenes = []
    for index, e in enumerate(test_path.iterdir()):
      #print(global_index)
      x_test[global_index] = read_img(e)
      y_test[global_index] = i
      global_index += 1
print(len(x_test), len(y_test))

# Cargamos una red entrenada
Vamos a cargar la red **ResNet50** ya entrenada, pero sin incluir las capas densas, ya que vamos a adaptar la red a nuestro caso específico.

In [ ]:
from keras.applications.resnet50 import ResNet50
base_model = ResNet50(include_top=False, input_shape=(img_size,img_size,3), pooling='avg')
base_model.summary()

/home/davicin/UtilsGeneral/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


66658304/94653016 [====================>.........] - ETA: 32s

In [ ]:
base_model.input, base_model.output

In [ ]:
# Dado que solo queremos entrenar las capas densas del modelo que agregaremos
# en el siguiente paso, vamos a setear "trainable = False" para que los pesos
# de la red entrenada no cambien.
base_model.trainable = False

# Creamos el modelo clasificador

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.optimizers import Adam

lr = 0.01

top_model = Sequential([
    Dense(128, activation='relu', input_shape=(2048,)),
    Dense(1, activation='softmax')
])

top_model.compile(SGD(lr, momentum=0.9), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
top_model.summary()

# Juntamos los 2 modelos

In [ ]:
final_model = Sequential([base_model, top_model])

final_model.compile(SGD(lr, momentum=0.9),loss='sparse_categorical_crossentropy', metrics=['accuracy'])
final_model.summary()

# Entrenando en nuestro dataset

In [ ]:
# log = final_model.fit(x_train, y_train, batch_size=64, validation_data=[x_val, y_val])

La red de base que estamos usando (ResNet50) tiene bastantes capas y hacer todas estas operaciones toma un tiempo considerable, en CPU hacer este entrenamiento puede resultar impractico.
# Precompute
Si tenemos en cuenta que vamos a entrenar nuestro dataset un cierto número de épocas, en cada época se van a repetir exactamente las mismas operaciones en la misma data. Para no redundar, es bastante útil hacer un precompute de la data:
1. Pasar todas nuestras imagenes por la red base (ResNet50).
2. Guardamos los features extraidos.
3. Entrenamos las capas densas con los features extraidos.

In [ ]:
precomputed_train = base_model.predict(x_train, batch_size=128, verbose=1)
precomputed_train.shape

In [ ]:
precomputed_val = base_model.predict(x_val, batch_size=128, verbose=1)
precomputed_val.shape

# Entrenar a partir de los features extraidos

In [ ]:
# Ahora podemos usar un batch_size mas grande, ya que los features son mas pequeños
# que las imagenes.
log = top_model.fit(precomputed_train, y_train, epochs=5, batch_size=256, validation_data=[precomputed_val, y_val])

In [ ]:
def show_results(log):
    fig, axes = plt.subplots(1, 2, figsize=(14,4))
    ax1, ax2 = axes
    ax1.plot(log.history['loss'], label='train')
    ax1.plot(log.history['val_loss'], label='validation')
    ax1.set_xlabel('epoch'); ax1.set_ylabel('loss')
    ax2.plot(log.history['acc'], label='train')
    ax2.plot(log.history['val_acc'], label='validation')
    ax2.set_xlabel('epoch'); ax2.set_ylabel('accuracy')
    for ax in axes: ax.legend()

In [ ]:
show_results(log)

# Ejercicio: usando el modelo completo en el test set

In [ ]:
test_path = Path('../input/test/')
test_files = list(test_path.iterdir())

In [ ]:
def get_class(path):
    # Cargar la imagen del path
    img = Image.open(path)
    
    # Cambiar el tamaño de la imagen
    img_resized = img.resize((224, 224))
    
    # Cambiar a formato numpy y preprocesar
    x = np.asarray(img_resized, np.float32)[None]
    x = preprocess_input(x)
    
    # Obtener predicciones
    y = final_model.predict(x)
    
    # Decodear predicciones
    pred = 'cat' if y < 0.5 else 'dog'
    
    # Mostrar la imagen
    plt.imshow(img)
    plt.axis('off')
    plt.title(pred, size=14)
    
    return

In [ ]:
sample = np.random.choice(test_files)
get_class(sample)